In [1]:
# !pip install opencv-python
# !pip install pandas
# !pip install matplotlib

import cv2
import numpy as np
from numpy.core.numeric import True_
from numpy.lib.arraysetops import unique
import pandas as pd
# import glob
import os
from pandas import json_normalize
from os import getcwd, path
from yaml import SafeLoader, load
import datetime
import matplotlib.pyplot as plt
import yaml
import shutil
import json
from time import time

from video_utils_virat import VideoUtils
from video_utils_virat import YoloUtils

%matplotlib inline

In [2]:
# setup
dataset_dir_path = '../datasets/VIRAT/'
image_ext = '.jpg'
video_max_frames = 2000

#video
video_ext = '.mp4'
video_src_path = dataset_dir_path + 'Videos/Ground/'
ann_validate_path = dataset_dir_path + 'viratannotations/validate/'
ann_train_path = dataset_dir_path + 'viratannotations/train/'


In [3]:
files = os.listdir(video_src_path)
existing_video_lst = [f.split('.')[0] for f in files if f.endswith(video_ext)]

annotations_lst_1 = os.listdir(ann_train_path)
annotations_lst_2 = os.listdir(ann_validate_path)

a = annotations_lst_1
b = annotations_lst_2
c = a[:]
for i in b:
    if i not in a:
        c.append(i)


missing_lst = c[:]
for i in c:
    if i in existing_video_lst:
        missing_lst.remove(i)


print(f'train:{len(a)}, validate:{len(b)}, ann total:{len(c)}, missing_videos:{len(missing_lst)}')

missing_lst = [n.strip('VIRAT_S_') for n in missing_lst]

print(missing_lst)

train:64, validate:55, ann total:119, missing_videos:0
[]


In [4]:
# using annotations:
print("Loading annotations...")
def add_category_type(row):
  id = row['object_id']
  val = type_df.loc[type_df['id'] == id, 'category'].iloc[0]
  return val


Loading annotations...


In [5]:
# Create directory to store new video
for video_name in annotations_lst_2:
    start_time = time()
    video_dest_path = './processed/' + video_name + '/'
    saved_csv = video_dest_path + 'df_bbox.csv'
    ann_types_file = ann_validate_path + video_name + '/' + video_name + '.types.yml'
    ann_geom_file = ann_validate_path + video_name + '/' + video_name + '.geom.yml'

    if not os.path.exists(video_dest_path):
        os.makedirs(video_dest_path)

    if not os.path.exists(saved_csv):
        with open(ann_types_file) as yaml_file:
            yaml_contents = load(yaml_file, Loader=SafeLoader)
        yaml_df = json_normalize(yaml_contents)
        yaml_df
        for col in yaml_df.columns:
            type_name = col.split('.')[-1]
            if not (type_name == 'id1'):
                yaml_df.loc[yaml_df[col] == 1, col] = type_name
        
        yaml_df = yaml_df[yaml_df['types.id1'].notna()].reset_index().dropna(axis=1, how='all')  
        type_df = yaml_df.ffill(axis=1).iloc[:,-1].to_frame(name='category')
        type_df.insert(0, "id", yaml_df['types.id1'])



    if os.path.exists(saved_csv):
        df_bbox = pd.read_csv(saved_csv)
    else:
        with open(ann_geom_file) as yaml_file:
            yaml_contents = load(yaml_file, Loader=SafeLoader)
        yaml_df = json_normalize(yaml_contents)

        df_bbox = yaml_df[['geom.id1','geom.ts0','geom.ts1','geom.g0']].dropna().reset_index()
        df_bbox.rename(columns={'geom.id1': 'object_id', 'geom.ts0': 'frame_id','geom.ts1': 'time_sec', 'geom.g0': 'bbox'}, inplace=True)
        df_bbox['bbox'] = df_bbox['bbox'].str.split()
        df_tmp = pd.DataFrame(df_bbox['bbox'].to_list(), columns = ['bb_left', 'bb_top', 'bb_right', 'bb_bottom'])
        df_bbox = pd.concat([df_bbox, df_tmp], axis=1).drop(columns=['bbox'])

        df_bbox['category'] = df_bbox.apply(lambda row: add_category_type(row), axis=1) 
        df_bbox.drop(columns=['index'], axis=1, inplace=True)
        # df_bbox.set_index['index'] 
        df_bbox.to_csv(saved_csv, index = False)
    
    dt = time() - start_time
    if dt > 3:
        print(f'Done with: {video_name}, {dt :0.2f} sec')
    
    df_bbox.head()

print(f'Done!!!')

Done!!!
